In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [7]:
import os
import shutil
from sklearn.model_selection import train_test_split

dataset_path = r"D:/VII SEM/Honors_project/WasteImagesDataset"

# paths for the training and testing sets
train_path = r'D:/VII SEM/Honors_project/train'
test_path = r'D:/VII SEM/Honors_project/test'

# Create train and test directories if not already present
os.makedirs(train_path, exist_ok=True)
os.makedirs(test_path, exist_ok=True)

# Split the dataset into train and test sets
class_folders = os.listdir(dataset_path)

for class_folder in class_folders:
    class_path = os.path.join(dataset_path, class_folder)
    train_class_path = os.path.join(train_path, class_folder)
    test_class_path = os.path.join(test_path, class_folder)

    # Create train and test class directories
    os.makedirs(train_class_path, exist_ok=True)
    os.makedirs(test_class_path, exist_ok=True)

    # List all images in the class folder
    images = os.listdir(class_path)

    # Split the images into train and test
    train_images, test_images = train_test_split(images, test_size=0.2, random_state=42)

    # Move images to respective train and test folders
    for img in train_images:
        src_path = os.path.join(class_path, img)
        dest_path = os.path.join(train_class_path, img)
        shutil.copy(src_path, dest_path)

    for img in test_images:
        src_path = os.path.join(class_path, img)
        dest_path = os.path.join(test_class_path, img)
        shutil.copy(src_path, dest_path)


In [2]:
train_dir = 'D:/VII SEM/Honors_project/train'
test_dir = 'D:/VII SEM/Honors_project/test'

In [3]:
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)  #Shear transformation 20% with respect to image dimensions - height or width

test_datagen = ImageDataGenerator(rescale=1./255) #rescale scales the pixels values to the range 0 to 1 for faster model convergence

In [4]:
batch_size = 32 
img_size = (224, 224)

In [5]:
# Load the train data
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=img_size,
                                                    batch_size=batch_size,
                                                    class_mode='categorical')

# Load the test data
test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size=img_size,
                                                  batch_size=batch_size,
                                                  class_mode='categorical')



Found 2190 images belonging to 5 classes.
Found 540 images belonging to 5 classes.


# Basic CNN approach

In [15]:
# Define the model architecture
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(5, activation='softmax'))  # 5 classes for Carton, fiber,Polythene, Steel, wooden

# Compile the model
model.compile(optimizer='adam', 
              loss='categorical_crossentropy',
              metrics=['accuracy']) 

#Train the model
model.fit(train_generator,
          steps_per_epoch=train_generator.samples // batch_size,
          epochs=30,
          validation_data=test_generator,
          validation_steps=test_generator.samples // batch_size)

# Save the model
model.save('waste_material_classification_model.h5')



Epoch 1/30
54/54 [==============================] - 82s 668ms/step - loss: 1.9947 - accuracy: 0.2755 - val_loss: 1.7992 - val_accuracy: 0.3490
Epoch 2/30
54/54 [==============================] - 77s 632ms/step - loss: 1.5711 - accuracy: 0.3778 - val_loss: 1.4856 - val_accuracy: 0.3917
Epoch 3/30
54/54 [==============================] - 77s 637ms/step - loss: 1.4063 - accuracy: 0.4372 - val_loss: 1.4780 - val_accuracy: 0.4406
Epoch 4/30
54/54 [==============================] - 77s 632ms/step - loss: 1.3052 - accuracy: 0.4758 - val_loss: 1.3433 - val_accuracy: 0.4875
Epoch 5/30
54/54 [==============================] - 77s 632ms/step - loss: 1.2354 - accuracy: 0.5100 - val_loss: 1.3294 - val_accuracy: 0.5052
Epoch 6/30
54/54 [==============================] - 76s 628ms/step - loss: 1.1770 - accuracy: 0.5387 - val_loss: 1.4050 - val_accuracy: 0.5031
Epoch 7/30
54/54 [==============================] - 77s 636ms/step - loss: 1.0820 - accuracy: 0.5711 - val_loss: 1.4484 - val_accuracy: 0.5083

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 26, 26, 128)      0

In [33]:
train_loss, train_accuracy = model.evaluate(train_generator)
print(f'Training Accuracy: {train_accuracy * 100:.2f}%')

test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Testing Accuracy: {test_accuracy * 100:.2f}%')

Training Accuracy: 94.40%
Testing Accuracy: 75.68%


# Using ResNet 

In [11]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator



# Specify the target size (dimensions) for resizing
img_size = (224, 224)
batch_size = 32

# Use data augmentation for better generalization
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   validation_split=0.2)  # Use validation split for creating a validation set

test_datagen = ImageDataGenerator(rescale=1./255)

# Load and augment the training data
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=img_size,
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    subset='training')  

# Load and augment the validation data
validation_generator = train_datagen.flow_from_directory(train_dir,
                                                         target_size=img_size,
                                                         batch_size=batch_size,
                                                         class_mode='categorical',
                                                         subset='validation') 

# Load the test data
test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size=img_size,
                                                  batch_size=batch_size,
                                                  class_mode='categorical')

# Load pre-trained ResNet50 model (without the top layer)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

# Create a new model by adding custom layers on top of the pre-trained model
model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(5, activation='softmax'))

# Compile the model with a lower learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model using the training and validation generators
model.fit(train_generator,
          steps_per_epoch=train_generator.samples // batch_size,
          epochs=30,
          validation_data=validation_generator,
          validation_steps=validation_generator.samples // batch_size)

# Save the model
model.save('keras_Model.h5')


Found 1754 images belonging to 5 classes.
Found 436 images belonging to 5 classes.
Found 540 images belonging to 5 classes.
94765736/94765736 [==============================] - 22s 0us/step
Epoch 1/30
54/54 [==============================] - 82s 668ms/step - loss: 1.9947 - accuracy: 0.2755 - val_loss: 1.7992 - val_accuracy: 0.3490
Epoch 2/30
54/54 [==============================] - 77s 632ms/step - loss: 1.5711 - accuracy: 0.3778 - val_loss: 1.4856 - val_accuracy: 0.3917
Epoch 3/30
54/54 [==============================] - 77s 637ms/step - loss: 1.4063 - accuracy: 0.4372 - val_loss: 1.4780 - val_accuracy: 0.4406
Epoch 4/30
54/54 [==============================] - 77s 632ms/step - loss: 1.3052 - accuracy: 0.4758 - val_loss: 1.3433 - val_accuracy: 0.4875
Epoch 5/30
54/54 [==============================] - 77s 632ms/step - loss: 1.2354 - accuracy: 0.5100 - val_loss: 1.3294 - val_accuracy: 0.5052
Epoch 6/30
54/54 [==============================] - 76s 628ms/step - loss: 1.1770 - accuracy: 0

In [35]:
train_loss, train_accuracy = model.evaluate(train_generator)
print(f'Training Accuracy: {train_accuracy * 100:.2f}%')

test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Testing Accuracy: {test_accuracy * 100:.2f}%')

Training Accuracy: 90.50%
Testing Accuracy: 88.67%


In [13]:
from keras.models import load_model  
from PIL import Image, ImageOps  
import numpy as np


filePath = 'D:/VII SEM/Honors_project/train/Metal/metal.jpg'
def pred(filePath):
    np.set_printoptions(suppress=True)

    # Load the model
    model = load_model("keras_Model.h5", compile=False)

    # Load the labels
    class_names = open("labels.txt", "r").readlines()

    # Create the array of the right shape to feed into the keras model
    # The 'length' or number of images you can put into the array is
    # determined by the first position in the shape tuple, in this case 1
    data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

    # Replace this with the path to your image
    image = Image.open(filePath).convert("RGB")

    # resizing the image to be at least 224x224 and then cropping f rom the center
    size = (224, 224)
    image = image.resize(size,resample=Image.BICUBIC)
    # turn the image into a numpy array
    image_array = np.asarray(image)

    # Normalize the image
    normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

    # Load the image into the array
    data[0] = normalized_image_array

    # Predicts the model
    prediction = model.predict(data)
    index = np.argmax(prediction)
    class_name = class_names[index]
    confidence_score = prediction[0][index]

    # Print prediction and confidence score
    print("prediction: ",prediction)
    print("Index: ",index)
    print("Class:", class_name[2:], end="")
    print("Confidence Score:", confidence_score)
    return class_name[2:]

pred(filePath)


1/1 [==============================] - 3s 3s/step
prediction:  [[0.         0.00000081 0.99972326 0.00027532 0.00000056]]
Index:  2
Class: Metal
Confidence Score: 0.99972326


'Metal\n'

In [4]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator




# Load pre-trained ResNet50 model (with the top layer)
base_model = ResNet50(weights='imagenet', include_top=True, input_shape=(224, 224, 3))
base_model.summary()




Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

                                                                                                  
 conv2_block3_1_relu (Activatio  (None, 56, 56, 64)  0           ['conv2_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block3_2_conv (Conv2D)   (None, 56, 56, 64)   36928       ['conv2_block3_1_relu[0][0]']    
                                                                                                  
 conv2_block3_2_bn (BatchNormal  (None, 56, 56, 64)  256         ['conv2_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_2_relu (Activatio  (None, 56, 56, 64)  0           ['conv2_block3_2_bn[0][0]']      
 n)       

                                                                                                  
 conv3_block3_1_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block3_2_conv (Conv2D)   (None, 28, 28, 128)  147584      ['conv3_block3_1_relu[0][0]']    
                                                                                                  
 conv3_block3_2_bn (BatchNormal  (None, 28, 28, 128)  512        ['conv3_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block3_2_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block3_2_bn[0][0]']      
 n)       

                                                                                                  
 conv4_block2_1_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block2_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block2_1_relu (Activatio  (None, 14, 14, 256)  0          ['conv4_block2_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block2_2_conv (Conv2D)   (None, 14, 14, 256)  590080      ['conv4_block2_1_relu[0][0]']    
                                                                                                  
 conv4_block2_2_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block2_2_conv[0][0]']    
 ization) 

 conv4_block5_1_conv (Conv2D)   (None, 14, 14, 256)  262400      ['conv4_block4_out[0][0]']       
                                                                                                  
 conv4_block5_1_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block5_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block5_1_relu (Activatio  (None, 14, 14, 256)  0          ['conv4_block5_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block5_2_conv (Conv2D)   (None, 14, 14, 256)  590080      ['conv4_block5_1_relu[0][0]']    
                                                                                                  
 conv4_blo

                                                                  'conv5_block1_3_bn[0][0]']      
                                                                                                  
 conv5_block1_out (Activation)  (None, 7, 7, 2048)   0           ['conv5_block1_add[0][0]']       
                                                                                                  
 conv5_block2_1_conv (Conv2D)   (None, 7, 7, 512)    1049088     ['conv5_block1_out[0][0]']       
                                                                                                  
 conv5_block2_1_bn (BatchNormal  (None, 7, 7, 512)   2048        ['conv5_block2_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block2_1_relu (Activatio  (None, 7, 7, 512)   0           ['conv5_block2_1_bn[0][0]']      
 n)       

In [10]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50


# Load the pre-trained ResNet-50 model
model = ResNet50(weights='imagenet')

# Iterate through layers and print activation functions
for layer in model.layers:
    print(f"{layer.name}: {layer.activation.__name__ if hasattr(layer, 'activation') else None}")


input_8: None
conv1_pad: None
conv1_conv: linear
conv1_bn: None
conv1_relu: relu
pool1_pad: None
pool1_pool: None
conv2_block1_1_conv: linear
conv2_block1_1_bn: None
conv2_block1_1_relu: relu
conv2_block1_2_conv: linear
conv2_block1_2_bn: None
conv2_block1_2_relu: relu
conv2_block1_0_conv: linear
conv2_block1_3_conv: linear
conv2_block1_0_bn: None
conv2_block1_3_bn: None
conv2_block1_add: None
conv2_block1_out: relu
conv2_block2_1_conv: linear
conv2_block2_1_bn: None
conv2_block2_1_relu: relu
conv2_block2_2_conv: linear
conv2_block2_2_bn: None
conv2_block2_2_relu: relu
conv2_block2_3_conv: linear
conv2_block2_3_bn: None
conv2_block2_add: None
conv2_block2_out: relu
conv2_block3_1_conv: linear
conv2_block3_1_bn: None
conv2_block3_1_relu: relu
conv2_block3_2_conv: linear
conv2_block3_2_bn: None
conv2_block3_2_relu: relu
conv2_block3_3_conv: linear
conv2_block3_3_bn: None
conv2_block3_add: None
conv2_block3_out: relu
conv3_block1_1_conv: linear
conv3_block1_1_bn: None
conv3_block1_1_relu: